# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,2,1,507,True
1,0,2,2,441,True
2,0,2,3,583,True
3,0,2,4,599,True
4,0,2,5,548,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,474.0,414.444444
1,2,0,False,495.0,418.444444
2,4,3,True,474.0,516.000000
3,5,0,False,468.5,427.777778
4,6,0,False,490.5,417.111111
...,...,...,...,...,...
5408,9990,0,False,487.5,418.444444
5409,9992,0,False,483.0,419.777778
5410,9993,0,False,459.5,418.444444
5411,9994,0,False,479.5,418.777778


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,474.0,414.444444,69,1,1
1,2,0,0,495.0,418.444444,27,0,1
2,4,3,1,474.0,516.000000,66,0,2
3,5,0,0,468.5,427.777778,68,0,1
4,6,0,0,490.5,417.111111,63,1,1


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [12]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

[20.06.2023 10:21:33 | matcher | INFO]: Applying matching


Finding index
Finding index


[20.06.2023 10:21:34 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 488
[20.06.2023 10:21:34 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: 4925
[20.06.2023 10:21:34 | Faiss matcher | INFO]: Estimating quality of matching
[20.06.2023 10:21:34 | metrics | INFO]: Standardised mean difference: age           0.000135
is_male       0.000000
pre_spends    0.001463
dtype: float64
[20.06.2023 10:21:34 | metrics | INFO]: Standardised mean difference: age           0.004546
is_male       0.000000
pre_spends    0.002101
dtype: float64
[20.06.2023 10:21:34 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 1.0, 'is_male': 1.0, 'pre_spends': 0.999999999999974}
[20.06.2023 10:21:34 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.34366775239307146, 'is_male': 1.0, 'pre_spends': 0.13012163366437443}
[20.06.2023 10:21:34 | psi_pandas | INFO]: Creatin

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.180976,0.572283,0.0,99.059302,101.302650
ATC,100.199038,0.585052,0.0,99.052337,101.345740
ATT,99.998688,0.696466,0.0,98.633614,101.363762


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [9]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.219962,0.566478,0.0,99.109665,101.330259
ATC,100.242493,0.579008,0.0,99.107637,101.377350
ATT,99.992570,0.680957,0.0,98.657894,101.327246


In [9]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

TypeError: __init__() got an unexpected keyword argument 'df'

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [ ]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [ ]:
features_importance = model.lama_feature_select()
features_importance

In [ ]:
features = features_importance['Feature'].to_list()

In [ ]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

In [ ]:
model.matcher.df_matched

## 3. Results  
### 3.1 ATE, ATT, ATC

In [ ]:
# model.matcher.results
results

### 3.2 SMD, PSI, KS-test, repeats

In [ ]:
# matching quality result - SMD
model.quality_result

In [ ]:
# matching quality result - PSI
model.quality_result['psi']

In [ ]:
# matching quality result - KS-test

model.quality_result['ks_test']

In [ ]:
# matching quality result - repeats
model.quality_result['repeats']

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [ ]:
 
model.validate_result()
 

In [ ]:
model.matcher.df_matched